Primero se opta por importar las bibliotecas de trabajo y las dependencias necesarias. 
La presente tarea se realizó utilizando Firefox.

In [1]:
# Importando bibliotecas de trabajo
import pandas as pd   
import requests      
from bs4 import BeautifulSoup 
import numpy as np 
import warnings
import re
warnings.filterwarnings("ignore")
from IPython import get_ipython
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium
  !pip install cssselect

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  options = webdriver.ChromeOptions()
  options.add_argument('-headless')
  options.add_argument('-no-sandbox')
  options.add_argument('-disable-dev-shm-usage')
  #options.headless = True
  #options.add_argument("--window-size=1920,1200")
  driver = webdriver.Chrome('chromedriver',options=options)
else:
  import os 
  os.environ["PATH"] = '$PATH:.'
  #driver = webdriver.Chrome('C:/Users/usuari/Documents/masterM2/Data_Digital/chromedriver')
  driver = webdriver.Firefox()

En este trabajo se va a optar por hacer dos ejercicios:
1. El primero consiste en crear un Dataframe con los datos de las 1000 películas más taquilleras de todos los tiempos, para este se utilizará la página principal en las que salen los títulos. 
2. Se procede a utilizar Beautiful Soup para poder automatizar una gran cantidad de campos. 
3. El presente muestra cómo se vería a nivel de DF ya automatizado.

In [3]:
# Asignando la URL para buscar las peliculas por IMDB Top 1000
url = 'https://www.imdb.com/search/title/?groups=top_1000'

# Esperando la petiticion del HTTP 
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
# Creando listas para guardar los datos dentro de un df
Nombre_pelicula = []
Puntuacion = []
Anno = []
Descripcion = []
Duracion = []
Rating = []
Genero = []

In [5]:
#storing the meaningfull required data in the variable
movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

#calling one by one using for loop
for store in movie_data:
    name = store.h3.a.text
    Nombre_pelicula.append(name)
    
    anno_lanzamiento = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    Anno.append(anno_lanzamiento)
    
    duracion_pelicula = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    Duracion.append(duracion_pelicula)
    
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    Rating.append(rate)
    
    puntuacion_pelicula  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    Puntuacion.append(puntuacion_pelicula)
    
    describe = store.find_all('p', class_ = 'text-muted')
    descripcion_pelicula = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
    Descripcion.append(descripcion_pelicula)
    
    genero_pelicula   = store.find('span', class_ = 'genre').text.replace('\n', '') if store.find('span', class_ = 'genre') else '^^^^^^'
    Genero.append(genero_pelicula)
    
# Creando un DF usando la biblioteca de pandas
movie_DF = pd.DataFrame({'Nombre pelicula': Nombre_pelicula, 'Año': Anno, 'Duración': Duracion, 'Puntuación': Rating,  'Descripción': Descripcion, "Género": Genero})

In [6]:
# Visualizando datos
movie_DF.head(4)

,Nombre pelicula,Año,Duración,Puntuación,Descripción,Género
0,The Batman,2022,176,8.4,"When the Riddler, a sadistic serial killer, be...","Action, Crime, Drama"
1,Spider-Man: No Way Home,2021,148,8.5,"With Spider-Man's identity now revealed, Peter...","Action, Adventure, Fantasy"
2,Dune: Part One,2021,155,8.1,A noble family becomes embroiled in a war for ...,"Action, Adventure, Drama"
3,The Dark Knight,2008,152,9.1,When the menace known as the Joker wreaks havo...,"Action, Crime, Drama"


Aquí se puede ver que las 4 películas que es necesario utilizar para obtener sus atributos son:
1. The Batman. 
2. Spider-Man: No Way Home.
3. Dune: Part One. 
4. The Dark Night. 

A continuación se empezará a ingresar en cada uno de los links propios de las películas, ya que, muchos de los campos solicitados no se encuentran en la página principal y es necesario abrir cada link por aparte para extraer los datos. Esto se procederá a hacer para las siguientes 4 películas y se procederá a hacer un diccionario para guardar los datos.

Nota: cada link tiene la imagen de cada una de las portadas de las película, para verlo, solo es necesario copiar y pegar el link en el explorador. 

In [7]:
# Explorando la primera pelicula de la lista - The Batman -
driver.get("https://www.imdb.com/title/tt1877830/?ref_=adv_li_tt")
print(driver.page_source[:500])
html_text = driver.page_source
from lxml import html
from lxml import cssselect
tree = html.document_fromstring(html_text)
from urllib.request import urlopen
from bs4 import BeautifulSoup

#html = urlopen('https://www.imdb.com/title/tt1877830/?ref_=adv_li_tt')
bs = BeautifulSoup(html_text, 'html.parser')

<html xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml" class=" scriptsOn" lang="es-ES"><head><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31YXrY93hfL.js" crossorigin="anonymous"></script><meta name="viewport" content="width=device-width"><meta charset="utf-8"><title>The Batman (2022) - IMDb</title><meta name="description" content="The Batman: Dirigido por Matt Reeves. Con Robert Pattinson, Zoë Kravitz, Jeffrey Wright, Colin Fa


In [8]:
# Se capturan todos los atributos buscados
var1 = tree.cssselect(".sc-b73cd867-0")
var2 = tree.cssselect(".sc-52284603-0 > li:nth-child(1) > a:nth-child(1)")
var3 = tree.cssselect("section.ipc-page-section:nth-child(17) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2)")
var4 = tree.cssselect(".sc-94726ce4-4 > div:nth-child(1) > div:nth-child(1) > a:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
var5 = tree.cssselect(".sc-16ede01-2")
var6 = tree.cssselect("div.ipc-chip-list:nth-child(1)")
var7 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)")
var8 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(2) > div:nth-child(2) > ul:nth-child(1)")
var9 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(3) > div:nth-child(2) > ul:nth-child(1)")
var10 = tree.cssselect("section.ipc-page-section:nth-child(8) > div:nth-child(2) > div:nth-child(2)")
images = bs.find('img', {'src':re.compile('.jpg')})

prod_list = []

for v1, v2, v3, v4, v5, v6, v7, v8, v9, v10 in zip(var1, var2, var3, var4, var5, var6, var7, var8, var9, var10):
  auxTxt = {
      'Nombre película ':v1.text_content(),
      'Año': v2.text_content().replace('\n', ''),
      'Duracion': v3.text_content(),
      'Puntuacion': v4.text_content(),
      'Descripcion': v5.text_content().replace('\n', ''),
      'Genero': [v6[0].text_content(), v6[1].text_content(), v6[2].text_content()],
      'Director': v7.text_content(),
      'Guion': [v8[0].text_content(), v8[1].text_content(), v8[2].text_content()],
      'Reparto': [v9[0].text_content(), v9[1].text_content(), v9[2].text_content()],
      'Lista de Peliculas en "Mas de este estilo"': [v10[0][2].text_content(), v10[1][2].text_content(), v10[2][2].text_content(), v10[3][2].text_content()],
      'Imagen': images['src']}

  prod_list.append(auxTxt)

In [9]:
# Visualizando resultados
prod_list

[{'Año': '2022',
  'Descripcion': 'Cuando Enigma, un sádico asesino en serie, comienza a asesinar a las principales figuras políticas de Gotham, Batman se ve obligado a investigar la corrupción oculta de la ciudad y a cuestionar la participación de su familia.',
  'Director': 'Matt Reeves',
  'Duracion': '2 horas 56 minutos',
  'Genero': ['Acción', 'Crimen', 'Drama'],
  'Guion': ['Matt Reeves', 'Peter Craig', 'Bill Finger(Batman created by)'],
  'Imagen': 'https://m.media-amazon.com/images/M/MV5BMDdmMTBiNTYtMDIzNi00NGVlLWIzMDYtZTk3MTQ3NGQxZGEwXkEyXkFqcGdeQXVyMzMwOTU5MDk@._V1_QL75_UX190_CR0,0,190,281_.jpg',
  'Lista de Peliculas en "Mas de este estilo"': ['El caballero oscuro',
   'Batman Begins',
   'El caballero oscuro: La leyenda renace',
   'Spider-Man: No Way Home'],
  'Nombre película ': 'The Batman',
  'Puntuacion': '8,4',
  'Reparto': ['Robert Pattinson', 'Zoë Kravitz', 'Jeffrey Wright']}]

In [10]:
# Explorando la primera pelicula de la lista - Spiderman No Way Home -
driver.get("https://www.imdb.com/title/tt10872600/?ref_=adv_li_tt")
print(driver.page_source[:500])
html_text = driver.page_source
from lxml import html
from lxml import cssselect
tree = html.document_fromstring(html_text)
from urllib.request import urlopen
from bs4 import BeautifulSoup

#html = urlopen('https://www.imdb.com/title/tt1877830/?ref_=adv_li_tt')
bs = BeautifulSoup(html_text, 'html.parser')

<html xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml" class=" scriptsOn" lang="es-ES"><head><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31YXrY93hfL.js" crossorigin="anonymous"></script><meta name="viewport" content="width=device-width"><meta charset="utf-8"><title>Spider-Man: No Way Home (2021) - IMDb</title><meta name="description" content="Spider-Man: No Way Home: Dirigido por Jon Watts. Con Tom Holland, Zendaya, Benedict 


In [11]:
# Se capturan todos los atributos buscados
var1 = tree.cssselect(".sc-b73cd867-0")
var2 = tree.cssselect(".sc-52284603-0 > li:nth-child(1) > a:nth-child(1)")
var3 = tree.cssselect("section.ipc-page-section:nth-child(15) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2)")
var4 = tree.cssselect(".sc-94726ce4-4 > div:nth-child(1) > div:nth-child(1) > a:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
var5 = tree.cssselect(".sc-16ede01-2")
var6 = tree.cssselect("div.ipc-chip-list:nth-child(1)")
var7 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)")
var8 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(2) > div:nth-child(2) > ul:nth-child(1)")
var9 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(3) > div:nth-child(2) > ul:nth-child(1)")
var10 = tree.cssselect("section.ipc-page-section:nth-child(7) > div:nth-child(2) > div:nth-child(2)")
images = bs.find('img', {'src':re.compile('.jpg')})

prod_list = []

for v1, v2, v3, v4, v5, v6, v7, v8, v9, v10 in zip(var1, var2, var3, var4, var5, var6, var7, var8, var9, var10):
  auxTxt = {
      'Nombre película ':v1.text_content(),
      'Año': v2.text_content().replace('\n', ''),
      'Duracion': v3.text_content(),
      'Puntuacion': v4.text_content(),
      'Descripcion': v5.text_content().replace('\n', ''),
      'Genero': [v6[0].text_content(), v6[1].text_content(), v6[2].text_content()],
      'Director': v7.text_content(),
      'Guion': [v8[0].text_content(), v8[1].text_content(), v8[2].text_content()],
      'Reparto': [v9[0].text_content(), v9[1].text_content(), v9[2].text_content()],
      'Lista de Peliculas en "Mas de este estilo"': [v10[0][2].text_content(), v10[1][2].text_content(), v10[2][2].text_content(), v10[3][2].text_content()],
      'Imagen': images['src']}

  prod_list.append(auxTxt)

In [12]:
# Visualizando resultados
prod_list

[{'Año': '2021',
  'Descripcion': 'Con la identidad de Spider-Man ahora revelada, Peter recurre al Doctor Strange en busca de ayuda. Algo sale mal y el multiverso se convierte en la mayor amenaza. Ahora Peter debe descubrir qué significa realmente ser Spider-Man.',
  'Director': 'Jon Watts',
  'Duracion': '2 horas 28 minutos',
  'Genero': ['Acción', 'Aventura', 'Fantasía'],
  'Guion': ['Chris McKenna',
   'Erik Sommers',
   'Stan Lee(based on the Marvel comic book by)'],
  'Imagen': 'https://m.media-amazon.com/images/M/MV5BN2MyZjc3ODAtODhmYy00YzI5LTgyZTQtMWM1NzQ0MGVjMzFhXkEyXkFqcGdeQXVyMTA3MDk2NDg2._V1_QL75_UX190_CR0,2,190,281_.jpg',
  'Lista de Peliculas en "Mas de este estilo"': ['Spider-Man: Lejos de casa',
   'Vengadores: Endgame',
   'Spider-Man: Homecoming',
   'Vengadores: Infinity War'],
  'Nombre película ': 'Spider-Man: No Way Home',
  'Puntuacion': '8,5',
  'Reparto': ['Tom Holland', 'Zendaya', 'Benedict Cumberbatch']}]

In [13]:
# Explorando la primera pelicula de la lista - Dune -
driver.get("https://www.imdb.com/title/tt1160419/?ref_=adv_li_tt")
print(driver.page_source[:500])
html_text = driver.page_source
from lxml import html
from lxml import cssselect
tree = html.document_fromstring(html_text)
from urllib.request import urlopen
from bs4 import BeautifulSoup

#html = urlopen('https://www.imdb.com/title/tt1877830/?ref_=adv_li_tt')
bs = BeautifulSoup(html_text, 'html.parser')

<html xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml" class=" scriptsOn" lang="es-ES"><head><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31YXrY93hfL.js" crossorigin="anonymous"></script><meta name="viewport" content="width=device-width"><meta charset="utf-8"><title>Dune (2021) - IMDb</title><meta name="description" content="Dune: Dirigido por Denis Villeneuve. Con Timothée Chalamet, Rebecca Ferguson, Oscar Isaac, Jason Momoa.


In [14]:
# Se capturan todos los atributos buscados
var1 = tree.cssselect(".sc-b73cd867-0")
var2 = tree.cssselect("ul.ipc-inline-list--show-dividers:nth-child(2) > li:nth-child(1) > a:nth-child(1)")
var3 = tree.cssselect("section.ipc-page-section:nth-child(17) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2)")
var4 = tree.cssselect(".sc-94726ce4-4 > div:nth-child(1) > div:nth-child(1) > a:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
var5 = tree.cssselect(".sc-16ede01-2")
var6 = tree.cssselect("div.ipc-chip-list:nth-child(1)")
var7 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)")
var8 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(2) > div:nth-child(2) > ul:nth-child(1)")
var9 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(3) > div:nth-child(2) > ul:nth-child(1)")
var10 = tree.cssselect("section.ipc-page-section:nth-child(8) > div:nth-child(2) > div:nth-child(2)")
images = bs.find('img', {'src':re.compile('.jpg')})

prod_list = []

for v1, v2, v3, v4, v5, v6, v7, v8, v9, v10 in zip(var1, var2, var3, var4, var5, var6, var7, var8, var9, var10):
  auxTxt = {
      'Nombre película ':v1.text_content(),
      'Año': v2.text_content().replace('\n', ''),
      'Duracion': v3.text_content(),
      'Puntuacion': v4.text_content(),
      'Descripcion': v5.text_content().replace('\n', ''),
      'Genero': [v6[0].text_content(), v6[1].text_content(), v6[2].text_content()],
      'Director': v7.text_content(),
      'Guion': [v8[0].text_content(), v8[1].text_content(), v8[2].text_content()],
      'Reparto': [v9[0].text_content(), v9[1].text_content(), v9[2].text_content()],
      'Lista de Peliculas en "Mas de este estilo"': [v10[0][2].text_content(), v10[1][2].text_content(), v10[2][2].text_content(), v10[3][2].text_content()],
      'Imagen': images['src']}

  prod_list.append(auxTxt)


In [15]:
# Visualizando resultados
prod_list

[{'Año': '2021',
  'Descripcion': 'Adaptación de la novela de ciencia ficción de Frank Herbert sobre el hijo de una familia noble que trata de vengarse de la muerte de su padre y al mismo tiempo salvar un planeta que se le ha encomendado proteger.',
  'Director': 'Denis Villeneuve',
  'Duracion': '2 horas 35 minutos',
  'Genero': ['Acción', 'Aventura', 'Drama'],
  'Guion': ['Jon Spaihts(guión de)',
   'Denis Villeneuve(guión de)',
   'Eric Roth(guión de)'],
  'Imagen': 'https://m.media-amazon.com/images/M/MV5BMmE1ZTA3MjQtY2E1My00NzM0LWIzN2EtMWJlZjc5ZTgzZTkwXkEyXkFqcGdeQXVyODIyOTEyMzY@._V1_QL75_UX190_CR0,0,190,281_.jpg',
  'Lista de Peliculas en "Mas de este estilo"': ['No mires arriba',
   'Free Guy',
   'Sin tiempo para morir',
   'Joker'],
  'Nombre película ': 'Dune',
  'Puntuacion': '8,1',
  'Reparto': ['Timothée Chalamet', 'Rebecca Ferguson', 'Zendaya']}]

In [16]:
# Explorando la primera pelicula de la lista - The Dark Knight -
driver.get("https://www.imdb.com/title/tt0468569/?ref_=adv_li_tt")
print(driver.page_source[:500])
html_text = driver.page_source
from lxml import html
from lxml import cssselect
tree = html.document_fromstring(html_text)
from urllib.request import urlopen
from bs4 import BeautifulSoup

#html = urlopen('https://www.imdb.com/title/tt1877830/?ref_=adv_li_tt')
bs = BeautifulSoup(html_text, 'html.parser')

<html xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml" class=" scriptsOn" lang="es-ES"><head><script async="" src="https://images-na.ssl-images-amazon.com/images/I/31YXrY93hfL.js" crossorigin="anonymous"></script><meta name="viewport" content="width=device-width"><meta charset="utf-8"><title>El caballero oscuro (2008) - IMDb</title><meta name="description" content="El caballero oscuro: Dirigido por Christopher Nolan. Con Christian Bale, Heath Ledger, A


In [17]:
# Se capturan todos los atributos buscados
var1 = tree.cssselect(".sc-b73cd867-0")
var2 = tree.cssselect("ul.ipc-inline-list--show-dividers:nth-child(2) > li:nth-child(1) > a:nth-child(1)")
var3 = tree.cssselect("section.ipc-page-section:nth-child(15) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2)")
var4 = tree.cssselect(".sc-94726ce4-4 > div:nth-child(1) > div:nth-child(1) > a:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)")
var5 = tree.cssselect(".sc-16ede01-2")
var6 = tree.cssselect("div.ipc-chip-list:nth-child(1)")
var7 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)")
var8 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(2) > div:nth-child(2) > ul:nth-child(1)")
var9 = tree.cssselect(".sc-fa02f843-0 > ul:nth-child(1) > li:nth-child(3) > div:nth-child(2) > ul:nth-child(1)")
var10 = tree.cssselect("section.ipc-page-section:nth-child(7) > div:nth-child(2) > div:nth-child(2)")
images = bs.find('img', {'src':re.compile('.jpg')})

prod_list = []

for v1, v2, v3, v4, v5, v6, v7, v8, v9, v10 in zip(var1, var2, var3, var4, var5, var6, var7, var8, var9, var10):
  auxTxt = {
      'Nombre película ':v1.text_content(),
      'Año': v2.text_content().replace('\n', ''),
      'Duracion': v3.text_content(),
      'Puntuacion': v4.text_content(),
      'Descripcion': v5.text_content().replace('\n', ''),
      'Genero': [v6[0].text_content(), v6[1].text_content(), v6[2].text_content()],
      'Director': v7.text_content(),
      'Guion': [v8[0].text_content(), v8[1].text_content(), v8[2].text_content()],
      'Reparto': [v9[0].text_content(), v9[1].text_content(), v9[2].text_content()],
      'Lista de Peliculas en "Mas de este estilo"': [v10[0][2].text_content(), v10[1][2].text_content(), v10[2][2].text_content(), v10[3][2].text_content()],
      'Imagen': images['src']}

  prod_list.append(auxTxt)

In [18]:
# Visualizando resultados
prod_list

[{'Año': '2008',
  'Descripcion': 'Cuando la amenaza conocida como el Joker causa estragos y el caos en Gotham City, Batman debe aceptar una de las mayores pruebas psicológicas y físicas para luchar contra la injusticia.',
  'Director': 'Christopher Nolan',
  'Duracion': '2 horas 32 minutos',
  'Genero': ['Acción', 'Crimen', 'Drama'],
  'Guion': ['Jonathan Nolan(guión)',
   'Christopher Nolan(guión)',
   'David S. Goyer(historia de)'],
  'Imagen': 'https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_QL75_UX190_CR0,0,190,281_.jpg',
  'Lista de Peliculas en "Mas de este estilo"': ['El caballero oscuro: La leyenda renace',
   'Batman Begins',
   'Joker',
   'Origen'],
  'Nombre película ': 'El caballero oscuro',
  'Puntuacion': '9,1',
  'Reparto': ['Christian Bale', 'Heath Ledger', 'Aaron Eckhart']}]